In [ ]:
# default_exp core

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# exports
def print_name(string):
    print(string)

In [ ]:
print_name('hello')

hello
